In [1]:
#import all the required libraries
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.preprocessing import image
from scipy.misc import imresize
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
#load the image and label data from the csv files (khanan's camera pictures from dentist's office)
train_image_path="DentalShades/Train/Images/"
train_file = "DentalShades/Train/train.csv"

train=pd.read_csv(train_file)

#loading the RGB data of train images into an array
train_img=[]
for i in range(len(train)):
    temp_img=image.load_img(train_image_path+train['filename'][i],target_size=(224,224))
    temp_img=image.img_to_array(temp_img)
    train_img.append(temp_img)

#converting test images to array and applying mean subtraction processing
train_img=np.array(train_img) 
train_img=preprocess_input(train_img)

In [3]:
# loading VGG16 model weights
vgg_model = VGG16(weights='imagenet', include_top=False)

# Extracting features from the train dataset using the VGG16 pre-trained model
features_train=vgg_model.predict(train_img)

# flattening the layers to conform to MLP input
train_x=features_train.reshape(len(train), 25088)

# converting target variable to array
train_y=np.asarray(train['label'])

# performing one-hot encoding for the target variable
train_y=pd.get_dummies(train_y)
train_y=np.array(train_y)

#X_train, X_valid, Y_train, Y_valid=train_test_split(train_x,train_y,test_size=0.3, random_state=42)

In [4]:
# creating a model to train on tooth shades
model=Sequential()
model.add(Dense(1000, input_dim=25088, activation='relu',kernel_initializer='uniform'))
keras.layers.core.Dropout(0.3, noise_shape=None, seed=None)
model.add(Dense(500,input_dim=1000,activation='sigmoid'))
keras.layers.core.Dropout(0.4, noise_shape=None, seed=None)
model.add(Dense(150,input_dim=500,activation='sigmoid'))
keras.layers.core.Dropout(0.2, noise_shape=None, seed=None)
model.add(Dense(units=26))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [5]:
model.fit(train_x, train_y, epochs=20, batch_size=13)
#model.fit(X_train, Y_train, epochs=100, batch_size=13,validation_data=(X_valid,Y_valid))

Epoch 1/20
26/26 [==============================] - 1s - loss: 3.5856 - acc: 0.0385     
Epoch 2/20
26/26 [==============================] - 0s - loss: 3.0891 - acc: 0.2692     
Epoch 3/20
26/26 [==============================] - 0s - loss: 2.9166 - acc: 0.4231     
Epoch 4/20
26/26 [==============================] - 0s - loss: 2.7880 - acc: 0.5769     
Epoch 5/20
26/26 [==============================] - 0s - loss: 2.7141 - acc: 0.8462     
Epoch 6/20
26/26 [==============================] - 0s - loss: 2.6410 - acc: 0.9231     
Epoch 7/20
26/26 [==============================] - 0s - loss: 2.5843 - acc: 0.9615     
Epoch 8/20
26/26 [==============================] - 1s - loss: 2.4977 - acc: 0.9615     
Epoch 9/20
26/26 [==============================] - 0s - loss: 2.4199 - acc: 1.0000     
Epoch 10/20
26/26 [==============================] - 0s - loss: 2.3365 - acc: 1.0000     
Epoch 11/20
26/26 [==============================] - 0s - loss: 2.2640 - acc: 0.9615     
Epoch 12/20
26/26 [

In [8]:
#testing the exact same data as the training data (khanan's camera pictures from dentist's office)
test_image_path="DentalShades/Test/Images/"
test_file = "DentalShades/Test/test.csv"

test=pd.read_csv(test_file)

test_img=[]
for i in range(len(test)):
    temp_img=image.load_img(test_image_path+train['filename'][i],target_size=(224,224))
    temp_img=image.img_to_array(temp_img)
    test_img.append(temp_img)

#converting test images to array and applying mean subtraction processing
test_img=np.array(test_img) 
test_img=preprocess_input(test_img)

# Extracting features from the train dataset using the VGG16 pre-trained model
features_test=vgg_model.predict(test_img)

test_x=features_test.reshape(len(test), 25088)
test_y=np.asarray(test['label'])
test_y=pd.get_dummies(test_y)
test_y=np.array(test_y)

score = model.evaluate(test_x, test_y, batch_size=26)
print(score)

26/26 [==============================] - 0s
[1.6020240783691406, 1.0]


In [ ]:
#testing on the internet data now